In [1]:
from bs4 import BeautifulSoup as bs
import requests
import os
import pandas as pd
from datetime import datetime
from IPython.display import clear_output

In [2]:
def convert_timedelta(duration):
    days, seconds = duration.days, duration.seconds
    hours = days * 24 + seconds // 3600
    minutes = (seconds % 3600) // 60
    seconds = (seconds % 60)
    print("Geçen zaman:")
    print(f"{hours} saat {minutes} dakika {seconds} saniye")

In [3]:
ilce_list = ["adalar","arnavutkoy","atasehir","avcilar","bagcilar","istanbul-bahcelievler","bakikoy","basaksehir","bayrampasa","besiktas","beykoz","beylikduzu","beyoglu","buyukcekmece","catalca","cekmekoy","esenler","esenyurt","eyupsultan","fatih","gaziosmanpasa","gungoren","kadikoy","kagithane","kartal","kucukcekmece","maltepe","pendik","sancaktepe","sariyer","sile","silivri","sisli","sultanbeyli","sultangazi","tuzla","umraniye","uskudar","zeytinburnu"]

In [4]:
start = datetime.now()
city_urls = []
pages = []
num_scraped_pages = 0

for city in ilce_list:
    url="https://www.zingat.com/" + city +"-satilik-daire?page="
    city_urls.append(url)

for i in range(0,len(city_urls)):
    response = requests.get(city_urls[i]).text
    soup = bs(response, 'html.parser')
    try:
        num_pages = soup.find("nav", "zng-pagination")['data-total']
        for j in range(1, int(num_pages)+1):
            page = city_urls[i] + str(j)
            pages.append(page)
            num_scraped_pages+=1
            
            clear_output(wait=True)
            print(f"Toplam {num_scraped_pages} sayfa çekildi.\n")
            convert_timedelta(datetime.now()-start)
    except:
        continue
        


Toplam 806 sayfa çekildi.

Geçen zaman:
0 saat 0 dakika 51 saniye


In [5]:
start = datetime.now()
page_url = []
cont_url = []
num_scraped_pages = 0
num_scraped_cont = 0

for page in pages:
    response = requests.get(page)
    soup = bs(response.text, "html.parser")
    page_url.append(soup.select(".zl-card-inner"))
    num_scraped_pages+=1
    
    clear_output(wait=True)
    print(f"Toplam {num_scraped_pages} sayfadan ilan çekildi.\n")
    convert_timedelta(datetime.now()-start)
    
for page in page_url:
    for url in page:
        cont_url.append("https://www.zingat.com" + url.get("href"))        
        num_scraped_cont+=1

print(f"\nÇekilen toplam ilan sayısı = {num_scraped_cont}")

Toplam 806 sayfadan ilan çekildi.

Geçen zaman:
0 saat 16 dakika 53 saniye

Çekilen toplam ilan sayısı = 16554


In [6]:
start = datetime.now()
detay_raw = []
oda= []
fiyat = []
mahalle = []
ilce =[]
il = []
num_scraped_cont = 0

for url in cont_url:
    try:
        response = requests.get(url)
        soup = bs(response.text, "html.parser")
        
        detay_raw.append([tag.text.strip() for tag in soup.find_all(["strong", "span"], {"class": "col-md-6"})])
        fiyat.append(soup.find("strong", itemprop = "price").text.replace("TL", "").replace(".","").replace(" ",""))
        oda.append(soup.find("div","detail-info not-printable").text.replace("\n",",").split(",")[5])
        mahalle.append(soup.find("div", "detail-location-path__map").find("h2").text.replace("\n","").split(",")[0].strip())
        ilce.append(soup.find("div", "detail-location-path__map").find("h2").text.replace("\n","").split(",")[1].strip())
        il.append(soup.find("div", "detail-location-path__map").find("h2").text.replace("\n","").split(",")[2].strip())
        num_scraped_cont+=1
        
        clear_output(wait=True)
        print(f"Toplam {num_scraped_cont} içerik çekildi.\n")    
        convert_timedelta(datetime.now()-start)
    except:
        continue
    
detay = []
for d in detay_raw:
    detay.append(dict(zip(d[::2], d[1::2])))

Toplam 16551 içerik çekildi.

Geçen zaman:
2 saat 43 dakika 11 saniye


In [7]:
df = pd.DataFrame(detay)
df['Fiyat'] = pd.Series(fiyat)
df['İl'] = pd.Series(il)
df['İlçe'] = pd.Series(ilce)
df['Mahalle'] = pd.Series(mahalle)
df['Oda'] = pd.Series(oda)

In [8]:
df.drop(["İlan no","Oda Sayısı","Banyo Sayısı","Konut Şekli","Aidat","Kira Getirisi",
         "Takasa Uygun","Kimden","Tapu Durumu","Yapının Durumu",
         "Görüntülü Ev Gezintisi\n                                                                ?"],
        axis=1,inplace=True)

In [11]:
df.to_csv("../data/zingat_ist_raw.csv")